In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.2 MB/s eta 0:00:00


In [ ]:
#import the libraries
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import openai
import io
from statistics import mean

## Load the Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('drive/MyDrive/challenge1_data.csv')

In [ ]:
df

# Fine tuning

In [ ]:
def get_composition(compound_id, braking_id, df):
  material_columns = ['A_1', 'C_1', 'B_1', 'C_2',
       'F_1', 'B_2', 'E_1', 'B_3', 'B_4', 'F_2', 'F_3', 'B_5', 'C_3', 'F_4',
       'F_5', 'B_6', 'B_7', 'B_8', 'A_2', 'A_3', 'B_9', 'B_10', 'C_4', 'B_11',
       'B_12', 'E_2', 'E_3', 'E_4', 'C_5', 'B_13', 'F_6', 'C_6', 'B_14',
       'B_15', 'E_5', 'B_16', 'A_4', 'F_7', 'F_8', 'E_6', 'E_7', 'B_17',
       'B_18', 'B_19', 'E_8', 'F_9', 'B_20', 'F_10', 'B_21', 'A_5', 'E_9',
       'F_11', 'F_12', 'E_10', 'B_22', 'D_1', 'E_11', 'B_23', 'B_24', 'A_6']
  composition_df = df[material_columns]
  composition = ""
  for index, row in composition_df.iterrows():
    for material_column in material_columns:
      if row[material_column] > 0:
        composition += str(row[material_column]) + "% " + material_column + ", "
    if composition[-1] == ' ':
      composition = composition[:-1]
    if composition[-1] == ',':
      composition = composition[:-1]
    break
  return composition

def get_user_prompt(compound_id, braking_id, df):
  material_composition = get_composition(compound_id, braking_id, df)
  prompt = "You have a compound with this composition of materials: " + material_composition + ". "
  prompt += "I want you to output the performance data which has the mu, pressure, time and speed values for a given time series. "
  prompt += "Remember that Mu is the most important value and should always follow the constraints and be as accurate as possible. "
  prompt += "Mu value should be approximately 0.6 with an allowed error of 0.1. "
  prompt += "What would be the performance data for this material with the above composition for the braking id '''" + braking_id + "'''?"
  return prompt

def get_assistant_prompt(compound_id, braking_id, df):
  prompt = ""
  prompt += "This is the performance data for braking id '''" + braking_id + "'''. "
  cols_to_measure = ['mu', 'time', 'pressure_bar', 'temp_c', 'speed_kph']
  new_df = df[cols_to_measure]
  new_df.rename(columns={"pressure_bar": "pressure", "speed_kph": "speed", "temp_c": "temp"})
  new_df = new_df.round(5)
  prompt += "This is the csv data: "
  prompt += new_df.to_csv(index=False)
  return prompt


def get_system_prompt():
  return "You are a highly skilled statistician from Harvard University who works at Brembo, where you specialize in performance braking systems and components as well as conducting research on braking systems. Given a friction material’s composition, you craft compelling synthetic performance data for a user given braking test type. The braking id will be delimited by triple quotes. You understand the importance of data analysis and seamlessly incorporate it for generating synthetic performance data based on historical performance data provided. You have a knack for paying attention to detail and curating synthetic data that is in line with the trends seen in the time series data you will be provided with. You are well versed with data and business analysis and use this knowledge for crafting the synthetic data."

## Testing

In [ ]:
def get_composition2(df):
  material_columns = ['E_6', 'E_5', 'E_1', 'F_9', 'F_2', 'C_1', 'B_21', 'B_19', 'B_23',
       'A_6', 'E_9', 'B_7', 'B_14', 'B_4', 'F_8', 'D_1', 'E_10', 'C_2',
       'F_12', 'B_10', 'B_2', 'E_7', 'B_17', 'B_8', 'B_12', 'C_3', 'B_6',
       'A_1', 'E_11', 'E_4', 'F_6', 'F_7']
  composition = ""
  for index, row in df.iterrows():
    for material_column in material_columns:
      if row[material_column] > 0:
        composition += str(row[material_column]) + "% " + material_column + ", "
    if composition[-1] == ' ':
      composition = composition[:-1]
    if composition[-1] == ',':
      composition = composition[:-1]
    break
  return composition

# **UTILS**

In [ ]:
# step_braking_id_map = {}
# for index, row in df[['step', 'braking_id']].iterrows():
#   step_braking_id_map[row['braking_id']] = row['step']
#   if index > 10000:
#     break
# print(step_braking_id_map)
step_braking_id_map = {'burnishing_1_1': 'Burnishing 1', 'burnishing_1_2': 'Burnishing 1', 'burnishing_1_3': 'Burnishing 1', 'burnishing_1_4': 'Burnishing 1', 'burnishing_1_5': 'Burnishing 1', 'burnishing_1_6': 'Burnishing 1', 'burnishing_1_7': 'Burnishing 1', 'burnishing_1_8': 'Burnishing 1', 'burnishing_1_9': 'Burnishing 1', 'burnishing_1_10': 'Burnishing 1', 'burnishing_1_11': 'Burnishing 1', 'burnishing_1_12': 'Burnishing 1', 'burnishing_1_13': 'Burnishing 1', 'burnishing_1_14': 'Burnishing 1', 'burnishing_1_15': 'Burnishing 1', 'burnishing_1_16': 'Burnishing 1', 'burnishing_1_17': 'Burnishing 1', 'characteristic_1_1': 'Characteristic 1', 'characteristic_1_2': 'Characteristic 1', 'characteristic_1_3': 'Characteristic 1', 'burnishing_2_1': 'Burnishing 2', 'burnishing_2_2': 'Burnishing 2', 'burnishing_2_3': 'Burnishing 2', 'burnishing_2_4': 'Burnishing 2', 'burnishing_2_5': 'Burnishing 2', 'characteristic_2_1': 'Characteristic 2', 'characteristic_2_2': 'Characteristic 2', 'characteristic_2_3': 'Characteristic 2', 'deceleration_series_1_1': 'Deceleration Series 1', 'deceleration_series_1_2': 'Deceleration Series 1', 'deceleration_series_1_3': 'Deceleration Series 1', 'deceleration_series_1_4': 'Deceleration Series 1', 'deceleration_series_1_5': 'Deceleration Series 1', 'deceleration_series_2_1': 'Deceleration Series 2', 'deceleration_series_2_2': 'Deceleration Series 2', 'deceleration_series_2_3': 'Deceleration Series 2', 'deceleration_series_2_4': 'Deceleration Series 2', 'deceleration_series_2_5': 'Deceleration Series 2', 'deceleration_series_3_1': 'Deceleration Series 3', 'deceleration_series_3_2': 'Deceleration Series 3', 'deceleration_series_3_3': 'Deceleration Series 3', 'deceleration_series_3_4': 'Deceleration Series 3', 'deceleration_series_3_5': 'Deceleration Series 3', 'deceleration_series_4_1': 'Deceleration Series 4', 'deceleration_series_4_2': 'Deceleration Series 4', 'deceleration_series_4_3': 'Deceleration Series 4', 'deceleration_series_4_4': 'Deceleration Series 4', 'deceleration_series_4_5': 'Deceleration Series 4', 'characteristic_3_1': 'Characteristic 3', 'characteristic_3_2': 'Characteristic 3', 'characteristic_3_3': 'Characteristic 3', 'fading_1_1': 'Fading 1', 'fading_1_2': 'Fading 1', 'fading_1_3': 'Fading 1', 'fading_1_4': 'Fading 1', 'fading_1_5': 'Fading 1', 'fading_1_6': 'Fading 1', 'fading_1_7': 'Fading 1', 'fading_1_8': 'Fading 1', 'fading_1_9': 'Fading 1', 'fading_1_10': 'Fading 1', 'fading_1_11': 'Fading 1', 'fading_1_12': 'Fading 1', 'fading_1_13': 'Fading 1', 'fading_1_14': 'Fading 1', 'fading_1_15': 'Fading 1', 'fading_1_16': 'Fading 1', 'fading_1_17': 'Fading 1', 'fading_1_18': 'Fading 1', 'fading_1_19': 'Fading 1', 'fading_1_20': 'Fading 1', 'characteristic_4_1': 'Characteristic 4', 'characteristic_4_2': 'Characteristic 4', 'characteristic_4_3': 'Characteristic 4', 'fading_2_1': 'Fading 2', 'fading_2_2': 'Fading 2', 'fading_2_3': 'Fading 2', 'fading_2_4': 'Fading 2', 'fading_2_5': 'Fading 2', 'fading_2_6': 'Fading 2', 'fading_2_7': 'Fading 2', 'fading_2_8': 'Fading 2', 'fading_2_9': 'Fading 2', 'fading_2_10': 'Fading 2', 'fading_2_11': 'Fading 2', 'fading_2_12': 'Fading 2', 'fading_2_13': 'Fading 2', 'fading_2_14': 'Fading 2', 'fading_2_15': 'Fading 2', 'fading_2_16': 'Fading 2', 'fading_2_17': 'Fading 2', 'fading_2_18': 'Fading 2', 'fading_2_19': 'Fading 2', 'fading_2_20': 'Fading 2', 'characteristic_5_1': 'Characteristic 5', 'characteristic_5_2': 'Characteristic 5', 'characteristic_5_3': 'Characteristic 5', 'increasing_temperature_1': 'Increasing temperature', 'increasing_temperature_2': 'Increasing temperature', 'increasing_temperature_3': 'Increasing temperature', 'increasing_temperature_4': 'Increasing temperature', 'increasing_temperature_5': 'Increasing temperature', 'increasing_temperature_6': 'Increasing temperature', 'increasing_temperature_7': 'Increasing temperature', 'increasing_temperature_8': 'Increasing temperature', 'deceleration_series_400_degrees_1': 'Deceleration Series 400 degrees', 'deceleration_series_400_degrees_2': 'Deceleration Series 400 degrees', 'deceleration_series_400_degrees_3': 'Deceleration Series 400 degrees', 'deceleration_series_400_degrees_4': 'Deceleration Series 400 degrees', 'deceleration_series_400_degrees_5': 'Deceleration Series 400 degrees', 'decreasing_temperature_1': 'Decreasing temperature', 'decreasing_temperature_2': 'Decreasing temperature', 'decreasing_temperature_3': 'Decreasing temperature', 'decreasing_temperature_4': 'Decreasing temperature', 'decreasing_temperature_5': 'Decreasing temperature', 'decreasing_temperature_6': 'Decreasing temperature', 'decreasing_temperature_7': 'Decreasing temperature', 'decreasing_temperature_8': 'Decreasing temperature', 'characteristic_6_1': 'Characteristic 6', 'characteristic_6_2': 'Characteristic 6', 'characteristic_6_3': 'Characteristic 6', 'characteristic_7_1': 'Characteristic 7', 'characteristic_7_2': 'Characteristic 7', 'characteristic_7_3': 'Characteristic 7'}

In [ ]:
def get_str(i):
  if i == 0:
    return "first"
  elif i == 1:
    return "second"
  elif i == 2:
    return "third"
  elif i == 3:
    return "fourth"
  elif i == 4:
    return "fifth"
  return "sixth"

cols_needed = ["mu", "time", "pressure_bar", "temp_c", "speed_kph"]
def get_perf_data_from_compound_id(compound_id, braking_id):
  ret = {}
  df_temp = pd.DataFrame(df[(df.compound_id==compound_id) & (df.braking_id==braking_id)])[cols_needed]
  return df_temp.round(3).to_csv(index=False)

def get_user_prompt_knn_context(compound_ids, braking_id):
  prompt = "Here are some similar compounds and their performance data you have to use for reference. "
  prompt += "Use them for generating the performance data. "

  i=0
  for compound_id in compound_ids:
    material_csv = get_perf_data_from_compound_id(compound_id, braking_id)
    prompt += "\n\nThe time series data for braking id " + braking_id + "  of the " + get_str(i) + " most similar compound is this:\n"
    prompt += get_perf_data_from_compound_id(compound_id, braking_id)
    i += 1

  prompt += "Notice that the value "
  prompt += "of mu is approximately 0.6. Strictly ensure that your output performance data "
  prompt += "also has the value of mu around 0.6 for all the time."
  return prompt

In [ ]:
def get_user_prompt_model(braking_id, row):
  compound_ids = row.neighbours.split("$")[0:4]
  material_composition = get_composition2(pd.DataFrame(row).T)
  prompt = "You have a compound with this composition of materials: " + material_composition + ". "
  prompt += "I want you to output the performance data which has the mu, pressure, time and speed values for a given time series. "
  prompt += get_user_prompt_knn_context(compound_ids, braking_id)
  prompt += "Remember that Mu is the most important value and should always follow the constraints and be as accurate as possible. "
  prompt += "Make sure that your average Mu is the same as the average Mu in above time series. Any mistake here is intolerable. "
  prompt += "Strictly ensure that you only generate 31 datapoints in the time series! Do not ever miss this. "
  prompt += "What would be the performance data for this material with the above composition for the braking id '''" + braking_id + "'''?"
  return prompt

In [ ]:
def get_column_names():
  return ",".join(list(df.columns)) + "\n"

import csv
import random
from io import StringIO

# Sample CSV string
def get_data_from_csv_string(csv_string):
  # Initialize an empty list to store the dictionaries
  data = []
  # Create a StringIO object to mimic a file
  csv_file = StringIO(csv_string)
  # Create a CSV reader that reads data into dictionaries
  csv_reader = csv.DictReader(csv_file)
  # Iterate over the rows in the CSV and convert them to dictionaries
  idx = 0
  for row in csv_reader:
    data.append(row)
    if idx == 0:
      columns_match = ['time', 'pressure_bar', 'temp_c', 'speed_kph', 'mu']
      columns = row.keys()
      if set(columns_match) != set(columns):
        return []
      idx = 1

  return data
# Now, 'data' contains a list of dictionaries, with each dictionary representing a row from the CSV
# print(get_data_from_csv_string(data_list[0]))

def add_rows(compound_id, braking_id, csv_strs, df_test, row_index, start_idx):
  csv_string = ""
  data_list = [json.content.split(":",1)[1].strip() for json in csv_strs]
  dict_datas = [get_data_from_csv_string(item) for item in data_list]
  column_list = list(df.columns)
  mat_columns = column_list[9:]
  # ['time_index', 'compound_id', 'step', 'braking_id', 'time', 'pressure_bar', 'temp_c', 'speed_kph', 'mu', 'A_1', 'C_1', 'B_1', 'C_2', 'F_1', 'B_2', 'E_1', 'B_3', 'B_4', 'F_2', 'F_3', 'B_5', 'C_3', 'F_4', 'F_5', 'B_6', 'B_7', 'B_8', 'A_2', 'A_3', 'B_9', 'B_10', 'C_4', 'B_11', 'B_12', 'E_2', 'E_3', 'E_4', 'C_5', 'B_13', 'F_6', 'C_6', 'B_14', 'B_15', 'E_5', 'B_16', 'A_4', 'F_7', 'F_8', 'E_6', 'E_7', 'B_17', 'B_18', 'B_19', 'E_8', 'F_9', 'B_20', 'F_10', 'B_21', 'A_5', 'E_9', 'F_11', 'F_12', 'E_10', 'B_22', 'D_1', 'E_11', 'B_23', 'B_24', 'A_6']

  for idx in range(len(csv_strs)):
    dict_data = dict_datas[idx]
    assert len(dict_data) >= 31

    for jdx in range(31):
      csv_string += f"{start_idx + jdx + 1}, new-compound{compound_id}, {step_braking_id_map[braking_id]}, {braking_id}, {dict_data[jdx]['time']}, {dict_data[jdx]['pressure_bar']}, {dict_data[jdx]['temp_c']}, {dict_data[jdx]['speed_kph']}, {dict_data[jdx]['mu']}"
      for mat_col in mat_columns:
        if mat_col in df_test.columns:
          csv_string += f", {df_test.iloc[row_index][mat_col]}"
        else:
          csv_string += ", 0"
      csv_string += "\n"
  return csv_string

def get_avg_mu(csv_strs):
  data_list = [json.content.split(":",1)[1].strip() for json in csv_strs]
  dict_datas = [get_data_from_csv_string(item) for item in data_list]
  for idx in range(len(csv_strs)):
    dict_data = dict_datas[idx]
    mus = [float(dict_data[j]['mu']) for j in range(31)]
    return mean(mus)

def is_float(element: any) -> bool:
    #If you expect None to be passed:
    if element is None:
        return False
    try:
        float(element)
        return True
    except ValueError:
        return False

def response_validate(csv_strs):
  split_str = csv_strs[0].content.split(":",1)
  if len(split_str) < 2:
    print("Problem with splitting " + csv_strs[0].content)
    return False
  data_list = [split_str[1].strip() for json in csv_strs]
  dict_datas = []
  for item in data_list:
    tmp = get_data_from_csv_string(item)
    if len(tmp) == 0:
      return False
    dict_datas.append(tmp)
  for idx in range(len(csv_strs)):
    dict_data = dict_datas[idx]
    if len(dict_data) < 31:
      print("Data len: " + str(len(dict_data)))
      return False
    for i in range(len(dict_data)):
      if not is_float(dict_data[i]['time']):
        print("Time not numeric " + dict_data[i]['time'])
        return False
      if not is_float(dict_data[i]['mu']):
        print("Mu not numeric " + dict_data[i]['mu'])
        return False
      if not is_float(dict_data[i]['speed_kph']):
        print("speed_kph not numeric " + dict_data[i]['speed_kph'])
        return False
      if not is_float(dict_data[i]['temp_c']):
        print("temp_c not numeric " + dict_data[i]['temp_c'])
        return False
      if not is_float(dict_data[i]['pressure_bar']):
        print("pressure_bar not numeric " + dict_data[i]['pressure_bar'])
        return False
  return True

def get_me_a_response(braking_id, row):
  openai.api_key = @key
  completion = openai.ChatCompletion.create(
    model="ft:gpt-3.5-turbo-0613:personal::89jkaokZ",
    messages=[
      {"role": "system", "content": get_system_prompt()},
      {"role": "user", "content": get_user_prompt_model(braking_id, row)}
    ]
  )
  responses.append(completion.choices[0].message)
  csv_strs = [completion.choices[0].message]
  retries = 0
  while response_validate(csv_strs) == False:
    completion = openai.ChatCompletion.create(
      model="ft:gpt-3.5-turbo-0613:personal::89jkaokZ",
      messages=[
        {"role": "system", "content": get_system_prompt()},
        {"role": "user", "content": get_user_prompt_model(braking_id, row)}
        ]
      )
    csv_strs = [completion.choices[0].message]
    retries += 1
    print("Retryingggg " + str(retries) + " braking id " + braking_id)
    if retries == 10:
      break
  return csv_strs

responses = []
def call_model_for_dataset(dataset_path):
  df_test = pd.read_csv(dataset_path)
  csv_string = ""
  for index, row in df_test.iterrows():  # Iterate over compounds
    print("Reched here")
    if index >= 10 and index <= 11:
      compound_id = random.randint(0,10000000)
      print("Iterating compound " + str(compound_id))
      tmp = 0
      start_idx = 0
      for braking_id in step_braking_id_map.keys():
        csv_strs = get_me_a_response(braking_id, row)
        # print("Mean is:" + str(get_avg_mu(csv_strs)))
        csv_string += add_rows(compound_id, braking_id, csv_strs, df_test, index, start_idx)
        start_idx += 31
        print("Called model for compound: " + str(compound_id) + " braking id: " + braking_id + " number " + str(tmp))
        tmp += 1
        # if tmp == 1:
          # break
  return csv_string


In [ ]:
csv_string = ""

csv_string += get_column_names()
csv_string += call_model_for_dataset('drive/MyDrive/model_generated_mats_subset_with_neighbours.csv')

print(csv_string)

Reched here
Iterating compound 2624505
Called model for compound: 2624505 braking id: burnishing_1_1 number 0
time_index,compound_id,step,braking_id,time,pressure_bar,temp_c,speed_kph,mu,A_1,C_1,B_1,C_2,F_1,B_2,E_1,B_3,B_4,F_2,F_3,B_5,C_3,F_4,F_5,B_6,B_7,B_8,A_2,A_3,B_9,B_10,C_4,B_11,B_12,E_2,E_3,E_4,C_5,B_13,F_6,C_6,B_14,B_15,E_5,B_16,A_4,F_7,F_8,E_6,E_7,B_17,B_18,B_19,E_8,F_9,B_20,F_10,B_21,A_5,E_9,F_11,F_12,E_10,B_22,D_1,E_11,B_23,B_24,A_6
1, new-compound2624505, Burnishing 1, burnishing_1_1, 0.95, 10.372, 30.25, 73.111, 0.509, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 17.3, 0.0, 0.0, 0.0, 0.0, 5.4, 0.0, 4.0, 0.0, 6.5, 0.0, 21.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 19.7, 0.0, 0.8, 0.0, 6.1, 0.0, 3.2
2, new-compound2624505, Burnishing 1, burnishing_1_1, 1.03783, 9.914, 31.532, 71.766, 0.524, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [ ]:
# print(csv_string)
# data = csv_string
# df = pd.DataFrame([x.split(',') for x in data.split('\n')])
df2 = pd.read_csv(io.StringIO(csv_string))
df2

# for braking_id in step_braking_id_map.keys():
#   normal_avg = df[df.braking_id==braking_id].mu.mean()
#   new_avg = df2[df2.braking_id==braking_id].mu.mean()
#   print(braking_id + ": " + str(normal_avg) + " " + str(new_avg))
# get_average_mus("17937-261", df, "new-compound3807681", df2)
# print(df2.groupby(['braking_id']).mu.mean())

,time_index,compound_id,step,braking_id,time,pressure_bar,temp_c,speed_kph,mu,A_1,C_1,B_1,C_2,F_1,B_2,E_1,B_3,B_4,F_2,F_3,B_5,C_3,F_4,F_5,B_6,B_7,B_8,A_2,A_3,B_9,B_10,C_4,B_11,B_12,E_2,E_3,E_4,C_5,B_13,F_6,C_6,B_14,B_15,E_5,B_16,A_4,F_7,F_8,E_6,E_7,B_17,B_18,B_19,E_8,F_9,B_20,F_10,B_21,A_5,E_9,F_11,F_12,E_10,B_22,D_1,E_11,B_23,B_24,A_6
0,1,new-compound5827131,Burnishing 1,burnishing_1_1,0.88500,10.133,27.291,72.542,0.459,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,17.3,0.0,0.0,0.0,0.0,5.4,0.0,4.0,0.0,6.5,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.7,0.0,0.8,0.0,6.1,0.0,3.2
1,2,new-compound5827131,Burnishing 1,burnishing_1_1,0.97567,9.79067,27.6406,71.329,0.462,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,17.3,0.0,0.0,0.0,0.0,5.4,0.0,4.0,0.0,6.5,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.7,0.0,0.8,0.0,6.1,0.0,3.2
2,3,new-compound5827131,Burnishing 1,burnishing_1_1,1.06633,9.453,28.13152,70.144,0.48067,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,17.3,0.0,0.0,0.0,0.0,5.4,0.0,4.0,0.0,6.5,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.7,0.0,0.8,0.0,6.1,0.0,3.2
3,4,new-compound5827131,Burnishing 1,burnishing_1_1,1.15700,9.294,28.49412,68.918,0.4895,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,17.3,0.0,0.0,0.0,0.0,5.4,0.0,4.0,0.0,6.5,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.7,0.0,0.8,0.0,6.1,0.0,3.2
4,5,new-compound5827131,Burnishing 1,burnishing_1_1,1.24767,9.151,29.10008,67.635,0.49117,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,17.3,0.0,0.0,0.0,0.0,5.4,0.0,4.0,0.0,6.5,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.7,0.0,0.8,0.0,6.1,0.0,3.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7683,3840,new-compound7531237,Characteristic 7,characteristic_7_3,6.33500,7.14793,153.94491,71.887,0.633,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,19.9,0.0,0.0,0.0,0.0,25.8,0.0,0.0,0.0,6.5,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,6.0,0.0,3.2
7684,3841,new-compound7531237,Characteristic 7,characteristic_7_3,6.56000,7.22316,155.73742,70.484,0.635,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,19.9,0.0,0.0,0.0,0.0,25.8,0.0,0.0,0.0,6.5,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,6.0,0.0,3.2
7685,3842,new-compound7531237,Characteristic 7,characteristic_7_3,6.78500,7.09276,157.591,69.085,0.624,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,19.9,0.0,0.0,0.0,0.0,25.8,0.0,0.0,0.0,6.5,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,6.0,0.0,3.2
7686,3843,new-compound7531237,Characteristic 7,characteristic_7_3,7.01000,7.37066,159.31732,67.71,0.617,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,19.9,0.0,0.0,0.0,0.0,25.8,0.0,0.0,0.0,6.5,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,6.0,0.0,3.2


In [ ]:
df2.to_csv('test_mantek.csv', index=False)